# Calibration script
In this script we calibrate the model parameters, by running for a range of different values and finding those which best agree with the experimental values. 

After we have found the parameters (which we will use for all other simulations), we also run a sweep of adhesion strength, K, to simulate the effect of the inhibitor

In [ ]:
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import h5py
import time
from IPython import display

from dedalus import public as de
from dedalus.extras import flow_tools

import logging
logger = logging.getLogger(__name__)

# Calibration of model parameters
Here we calibrate the parameters used in the simulations with physical values.

In [ ]:
import interface_probe as ip
Diff = np.linspace(2e-3,4e-3,20) # was 20
K =  0.0 # first run with no interfaces
v = 3.04/60 # mm per minute
v_min = 2.64/60 # mm per minute
v_max = 3.44/60 # mm per minute

w = np.zeros(np.size(Diff))
w_min = np.zeros(np.size(Diff))
w_max = np.zeros(np.size(Diff))
for i in range(np.size(Diff)):
    g = v*v/4/Diff[i]
    g_min = v_min*v_min/4/Diff[i]
    g_max = v_max*v_max/4/Diff[i]
    w[i] = ip.interface_width(Diff[i],K,g)
    w_min[i] = ip.interface_width(Diff[i],K,g_min)
    w_max[i] = ip.interface_width(Diff[i],K,g_max)

In [ ]:
exp_width = 1.29 #mm
exp_width_max = 1.31 #mm
exp_width_min = 1.27 #mm

f = interp1d(w, Diff)
f_min = interp1d(w_min, Diff)
f_max = interp1d(w_max, Diff)

Diff_max = f_min(exp_width_max).flat[0]
Diff_min = f_max(exp_width_min).flat[0]
Diff_fix = f(exp_width).flat[0]
print(Diff_max/60 * 1e6)
print(Diff_min/60 * 1e6)
print(Diff_fix/60 * 1e6)

In [ ]:
g = v*v/4/Diff_fix
g_max = v_max*v_max/4/Diff_min
g_min = v_min*v_min/4/Diff_max
print(g*60)
print(g_min*60)
print(g_max*60)

In [ ]:
K_scan =  np.linspace(0,3.4,25) 
w_int = np.zeros(np.size(K_scan))
w_int_min = np.zeros(np.size(K_scan))
w_int_max = np.zeros(np.size(K_scan))

for i in range(np.size(K_scan)):
    w_int[i] = ip.interface_width(Diff_fix,K_scan[i],g)
    w_int_min[i] = ip.interface_width(Diff_max,K_scan[i],g_min)
    w_int_max[i] = ip.interface_width(Diff_min,K_scan[i],g_max)

In [ ]:
fig, axis = plt.subplots(1,3,figsize=(18,5))
axis[0].plot(Diff/60 * 1e6,w,marker='o')
axis[0].plot(Diff/60 * 1e6,w_min,marker='o')
axis[0].plot(Diff/60 * 1e6,w_max,marker='o')
axis[0].plot(Diff/60 * 1e6,1.29*np.ones(np.shape(Diff)))
axis[0].set_xlabel("Diffusion coefficient (um^2/s)")
axis[0].set_ylabel("Interface width (mm)")

axis[1].plot(v*v/4/Diff *60,w,marker='o')
axis[1].plot(v_min*v_min/4/Diff *60,w_min,marker='o')
axis[1].plot(v_max*v_max/4/Diff *60,w_max,marker='o')
axis[1].plot([5,25],[1.3,1.3])
axis[1].set_xlabel("Growth rate (1/s)")
axis[1].set_ylabel("Interface width (mm)")

axis[2].plot(K_scan*60,w_int,marker='o')
axis[2].plot(K_scan*60,w_int_min,marker='o')
axis[2].plot(K_scan*60,w_int_max,marker='o')
#axis[2].plot(K_scan*60,np.sqrt(Diff_fix) *11/np.sqrt(g + 0.5*K_scan))
axis[2].plot(K_scan*60,0.5*np.ones(np.shape(K_scan)))
axis[2].set_xlabel("Inhibitor value (1/h)")
axis[2].set_ylabel("Interface width (mm)")
fig.savefig('Calibrate_sim.pdf',dpi=150)

In [ ]:
exp_int_width = 0.5 #mm
exp_int_width_max = 0.51 #mm
exp_int_width_min = 0.49 #mm

f_int = interp1d(w_int, K_scan)
f_int_min = interp1d(w_int_min, K_scan)
f_int_max = interp1d(w_int_max, K_scan)

K_max = f_int_max(exp_int_width_max).flat[0]
K_min = f_int_min(exp_int_width_max).flat[0]
K_fix = f_int(exp_int_width).flat[0]
print(K_min*60)
print(K_max*60)
print(K_fix*60)

In [ ]:
np.savez('Parameters.npz',K=K_fix,g=g,Diff = Diff_fix)

# Inhibitor calibrate

In [ ]:
import interface_probe as ip
params = np.load('Parameters.npz')
Diff = params['Diff'].flat[0]
I = np.exp(np.linspace(-8,5.8,30))# -8,5.3
Km =  params['K'].flat[0]
g = params['g'].flat[0]
w = np.zeros(np.size(I))
for i in range(np.size(I)):
    K = Km/(1+I[i])
    w[i] = ip.interface_width(Diff,K,g)

In [ ]:
np.savez('Inhibitor.npz',inh_conc=I*5e-6,int_width=w)